In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import rasterio
from tqdm import tqdm

# ========= PATH CONFIG =========
CSV_IN_ROOT  = Path("/mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full")
CSV_OUT_ROOT = Path("/mnt/cephfs-mount/chenchen/CygnssDataCsvLand_GWP")
GWP_ROOT     = Path("/mnt/cephfs-mount/chenchen/GlobalWaterPack")

# ========= GLOBAL WATERPACK AVAILABILITY =========
# Fixed valid date range for GWP product
GWP_START_DATE = dt.date(2019, 1, 1)
GWP_END_DATE   = dt.date(2019, 12, 31)

# ========= PERIOD TO PROCESS (EDIT IF NEEDED) =========
PROCESS_START_DATE = GWP_START_DATE
PROCESS_END_DATE   = GWP_END_DATE

# ========= SAMPLING CONFIG =========
SAMPLE_RATE = 0.03   # pre-sample 1% of rows from each CSV
RANDOM_SEED = 42     # random seed for reproducibility


def get_date_from_filename(csv_path: Path):
    """Extract YYYYMMDD from filename like '20180901.csv' → datetime.date."""
    stem = csv_path.stem
    if len(stem) < 8:
        return None
    try:
        d = dt.datetime.strptime(stem[:8], "%Y%m%d").date()
        return d
    except Exception:
        return None


def build_gwp_paths(date_obj: dt.date):
    """Build paths to GWP main & obs files for a given date."""
    y = date_obj.year
    m = date_obj.month
    d = date_obj.day
    ymd = f"{y:04d}{m:02d}{d:02d}"

    gwp_dir = GWP_ROOT / f"{y:04d}" / f"{m:02d}" / f"{d:02d}"
    stem = f"GWP.OSWF.DAILY.{ymd}.v1"

    main_path = gwp_dir / f"{stem}.tif"
    obs_path  = gwp_dir / f"{stem}.obs.tif"
    return main_path, obs_path


def compute_water_fraction_for_row(
    row,
    arr_main: np.ndarray,
    arr_obs: np.ndarray,
    src_main: rasterio.io.DatasetReader,
    obs_nodata,
) -> float:
    """
    Compute GWP-based water fraction for a single CYGNSS observation.

    Assumptions:
      - CSV coordinates are lon/lat (EPSG:4326).
      - GWP rasters are also EPSG:4326.
      - OBS semantics:
          1   → water observation
          101 → land observation
          102 → ocean surface observation
          others / nodata → not valid observation

    Steps:
      - Build footprint from c1..c4 lon/lat.
      - Convert footprint corners to raster row/col indices.
      - Extract bounding window for main and obs arrays.
      - Require ALL obs pixels in the window to be in {1, 101, 102} and != nodata
        (full-obs requirement).
      - Define water pixels as: main == 1 AND obs in {1, 102}.
      - water_fraction = water_pixels / total_pixels (pixel fraction).
    """
    try:
        lons = [
            float(row.c1_lon), float(row.c2_lon),
            float(row.c3_lon), float(row.c4_lon),
        ]
        lats = [
            float(row.c1_lat), float(row.c2_lat),
            float(row.c3_lat), float(row.c4_lat),
        ]
    except AttributeError as e:
        raise KeyError(f"Missing corner columns in row: {e}")

    rows = []
    cols = []
    for lon, lat in zip(lons, lats):
        try:
            # EPSG:4326 → index(lon, lat)
            r, c = src_main.index(lon, lat)
            rows.append(r)
            cols.append(c)
        except Exception:
            # Any corner outside the raster → discard this footprint
            return np.nan

    rmin = min(rows)
    rmax = max(rows)
    cmin = min(cols)
    cmax = max(cols)

    height, width = arr_main.shape
    if (rmax < 0 or rmin >= height or cmax < 0 or cmin >= width):
        # Entire footprint outside raster
        return np.nan

    rmin_c = max(rmin, 0)
    rmax_c = min(rmax, height - 1)
    cmin_c = max(cmin, 0)
    cmax_c = min(cmax, width - 1)

    if rmin_c > rmax_c or cmin_c > cmax_c:
        return np.nan

    sub_main = arr_main[rmin_c : rmax_c + 1, cmin_c : cmax_c + 1]
    sub_obs  = arr_obs[rmin_c : rmax_c + 1, cmin_c : cmax_c + 1]

    if sub_main.size == 0 or sub_obs.size == 0:
        return np.nan

    # ---- FULL OBS REQUIREMENT ----
    # Valid obs: 1 (water), 101 (land obs), 102 (ocean obs)
    valid_obs_mask = np.isin(sub_obs, (1, 101, 102))
    if obs_nodata is not None:
        valid_obs_mask &= (sub_obs != obs_nodata)

    # If any pixel in the footprint is not a valid observation, discard
    if not np.all(valid_obs_mask):
        return np.nan

    # ---- WATER FRACTION (pixel fraction) ----
    # Water pixels: main == 1 AND obs in {1, 102}
    water_mask = (sub_main == 1) & np.isin(sub_obs, (1, 102))
    if water_mask.size == 0:
        return np.nan

    frac = float(water_mask.mean())  # fraction in [0, 1]
    return frac


def process_one_csv(csv_in_path: Path):
    """Process a single daily CSV with GWP + pre-sampling + water fraction."""
    date_obj = get_date_from_filename(csv_in_path)
    if date_obj is None:
        print(f"[WARN] Could not parse date from filename: {csv_in_path.name}, skipping.")
        return

    # Filter by user-defined processing period
    if date_obj < PROCESS_START_DATE or date_obj > PROCESS_END_DATE:
        return

    # Also require date to be within GWP product availability
    if date_obj < GWP_START_DATE or date_obj > GWP_END_DATE:
        print(f"[INFO] {csv_in_path.name}: date {date_obj} outside GWP range, skipping.")
        return

    main_path, obs_path = build_gwp_paths(date_obj)
    if not main_path.exists() or not obs_path.exists():
        print(f"[WARN] Missing GWP files for {date_obj}:\n"
              f"   main: {main_path.exists()}  path={main_path}\n"
              f"   obs : {obs_path.exists()}  path={obs_path}\n"
              f"Skipping this CSV.")
        return

    # Output path (mirror directory structure)
    rel = csv_in_path.relative_to(CSV_IN_ROOT)
    csv_out_path = CSV_OUT_ROOT / rel
    csv_out_path.parent.mkdir(parents=True, exist_ok=True)

    print(f"\n[INFO] === Processing {csv_in_path} (date {date_obj}) ===")
    df = pd.read_csv(csv_in_path, low_memory=False)

    required_cols = [
        "c1_lon", "c2_lon", "c3_lon", "c4_lon",
        "c1_lat", "c2_lat", "c3_lat", "c4_lat",
    ]
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Missing required column '{col}' in {csv_in_path}")

    # ====== 1. Pre-sample rows from the CSV (before GWP lookup) ======
    n_total = len(df)
    n_keep  = max(int(n_total * SAMPLE_RATE), 1)
    df = df.sample(n=n_keep, random_state=RANDOM_SEED).reset_index(drop=True)
    print(f"[INFO] Pre-sampled rows: {n_total} → {len(df)} (rate ~{SAMPLE_RATE})")

    # ====== 2. Open GWP rasters and read into memory ======
    print(f"[INFO] Loading GWP rasters for {date_obj} ...")
    with rasterio.open(main_path) as src_main, rasterio.open(obs_path) as src_obs:
        if src_main.crs != src_obs.crs:
            raise ValueError(f"CRS mismatch between main and obs:\n"
                             f"  main: {src_main.crs}\n  obs : {src_obs.crs}")

        print(f"[INFO] Raster CRS: {src_main.crs}")

        arr_main = src_main.read(1)
        arr_obs  = src_obs.read(1)
        obs_nodata = src_obs.nodata

        print(f"[INFO] GWP shapes: main={arr_main.shape}, obs={arr_obs.shape}")
        print(f"[INFO] obs nodata: {obs_nodata}")

        n = len(df)
        water_frac = np.full(n, np.nan, dtype=np.float32)

        print(f"[INFO] Computing water_fraction_gwp for {n} sampled rows on {date_obj} ...")

        # No inner progress bar here (simple loop for each sampled row)
        for idx, row in enumerate(df.itertuples(index=False)):
            wf = compute_water_fraction_for_row(
                row,
                arr_main,
                arr_obs,
                src_main,
                obs_nodata,
            )
            water_frac[idx] = wf

    df["water_fraction_gwp"] = water_frac

    # ====== 3. Filter out footprints that do not meet full-obs requirement (NaN) ======
    before = len(df)
    df = df[~np.isnan(df["water_fraction_gwp"])].reset_index(drop=True)
    after = len(df)
    print(f"[INFO] Filtered rows by full-obs footprint: {before} → {after}")

    # ====== 4. Update water_fraction column ======
    if "water_fraction" in df.columns:
        if "water_fraction_orig" not in df.columns:
            df["water_fraction_orig"] = df["water_fraction"]
        df["water_fraction"] = df["water_fraction_gwp"]
        print("[INFO] Overwrote existing 'water_fraction' and backed it up to 'water_fraction_orig'.")
    else:
        df["water_fraction"] = df["water_fraction_gwp"]
        print("[INFO] Created new 'water_fraction' column from GWP.")

    print(f"[INFO] Writing output to: {csv_out_path}")
    df.to_csv(csv_out_path, index=False)


def main():
    print(f"[INFO] Input root         : {CSV_IN_ROOT}")
    print(f"[INFO] Output root        : {CSV_OUT_ROOT}")
    print(f"[INFO] GWP root           : {GWP_ROOT}")
    print(f"[INFO] GWP date range     : {GWP_START_DATE} → {GWP_END_DATE}")
    print(f"[INFO] Process date range : {PROCESS_START_DATE} → {PROCESS_END_DATE}")
    print(f"[INFO] Sampling rate      : {SAMPLE_RATE} (keep ~1 out of {int(1/SAMPLE_RATE)})")
    print()

    if not CSV_IN_ROOT.exists():
        print(f"[ERROR] Input root does not exist: {CSV_IN_ROOT}")
        return

    csv_files = sorted(CSV_IN_ROOT.rglob("*.csv"))
    if not csv_files:
        print("[ERROR] No CSV files found under input root.")
        return

    # Filter only CSV files whose dates fall within the processing period
    candidate_files = []
    for p in csv_files:
        d = get_date_from_filename(p)
        if d is None:
            continue
        if PROCESS_START_DATE <= d <= PROCESS_END_DATE:
            candidate_files.append(p)

    print(f"[INFO] Found {len(csv_files)} CSV files in total.")
    print(f"[INFO] {len(candidate_files)} CSV files in selected period.")
    print()

    # Outer progress bar: one step per day/CSV
    for csv_path in tqdm(candidate_files, desc="Days", unit="day"):
        try:
            process_one_csv(csv_path)
        except Exception as e:
            print(f"[ERROR] Failed to process {csv_path}: {e}")

    print("\n[INFO] All done.")


# In a Jupyter notebook, just run this cell to execute:
main()


[INFO] Input root         : /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full
[INFO] Output root        : /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_GWP
[INFO] GWP root           : /mnt/cephfs-mount/chenchen/GlobalWaterPack
[INFO] GWP date range     : 2019-01-01 → 2019-12-31
[INFO] Process date range : 2019-01-01 → 2019-12-31
[INFO] Sampling rate      : 0.03 (keep ~1 out of 33)

[INFO] Found 2526 CSV files in total.
[INFO] 365 CSV files in selected period.



Days:   0%|                                                                                                          | 0/365 [00:00<?, ?day/s]


[INFO] === Processing /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2019_01_v3/20190101.csv (date 2019-01-01) ===
[INFO] Pre-sampled rows: 753651 → 22609 (rate ~0.03)
[INFO] Loading GWP rasters for 2019-01-01 ...
[INFO] Raster CRS: EPSG:4326
[INFO] GWP shapes: main=(86400, 172800), obs=(86400, 172800)
[INFO] obs nodata: 255.0
[INFO] Computing water_fraction_gwp for 22609 sampled rows on 2019-01-01 ...
[INFO] Filtered rows by full-obs footprint: 22609 → 19797
[INFO] Created new 'water_fraction' column from GWP.
[INFO] Writing output to: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_GWP/2019_01_v3/20190101.csv


Days:   0%|▎                                                                                             | 1/365 [06:01<36:32:23, 361.38s/day]


[INFO] === Processing /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2019_01_v3/20190102.csv (date 2019-01-02) ===
[INFO] Pre-sampled rows: 753787 → 22613 (rate ~0.03)
[INFO] Loading GWP rasters for 2019-01-02 ...
[INFO] Raster CRS: EPSG:4326
[INFO] GWP shapes: main=(86400, 172800), obs=(86400, 172800)
[INFO] obs nodata: 255.0
[INFO] Computing water_fraction_gwp for 22613 sampled rows on 2019-01-02 ...
[INFO] Filtered rows by full-obs footprint: 22613 → 19825
[INFO] Created new 'water_fraction' column from GWP.
[INFO] Writing output to: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_GWP/2019_01_v3/20190102.csv


...


[INFO] === Processing /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2019_12_v3/20191230.csv (date 2019-12-30) ===
[INFO] Pre-sampled rows: 1496454 → 44893 (rate ~0.03)
[INFO] Loading GWP rasters for 2019-12-30 ...
[INFO] Raster CRS: EPSG:4326
[INFO] GWP shapes: main=(86400, 172800), obs=(86400, 172800)
[INFO] obs nodata: 255.0
[INFO] Computing water_fraction_gwp for 44893 sampled rows on 2019-12-30 ...
[INFO] Filtered rows by full-obs footprint: 44893 → 39488
[INFO] Created new 'water_fraction' column from GWP.
[INFO] Writing output to: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_GWP/2019_12_v3/20191230.csv


Days: 100%|███████████████████████████████████████████████████████████████████████████████████████████▋| 364/365 [21:04:42<02:15, 135.14s/day]


[INFO] === Processing /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_full/2019_12_v3/20191231.csv (date 2019-12-31) ===
[INFO] Pre-sampled rows: 1505235 → 45157 (rate ~0.03)
[INFO] Loading GWP rasters for 2019-12-31 ...
[INFO] Raster CRS: EPSG:4326
[INFO] GWP shapes: main=(86400, 172800), obs=(86400, 172800)
[INFO] obs nodata: 255.0
[INFO] Computing water_fraction_gwp for 45157 sampled rows on 2019-12-31 ...
[INFO] Filtered rows by full-obs footprint: 45157 → 39757
[INFO] Created new 'water_fraction' column from GWP.
[INFO] Writing output to: /mnt/cephfs-mount/chenchen/CygnssDataCsvLand_GWP/2019_12_v3/20191231.csv


Days: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [21:06:52<00:00, 208.25s/day]


[INFO] All done.
